# PySpark Warmup

## Installation for PySpark




In [1]:
!apt -y install openjdk-8-jre-headless
%pip install pyspark==3.0.1

/bin/bash: /home/s311511038/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: /home/s311511038/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


#### Start a simple Spark Session

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType

spark = SparkSession.builder.appName('Lab3').getOrCreate()

Data Schema

In [ ]:
data_schema = [
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True)
]
final_struc = StructType(fields=data_schema)


Load the people.json gile, have Spark infer the data types.

In [ ]:
df = spark.read.json('people.json', schema=final_struc)


#### What are the column names?

In [ ]:
df.columns

#### What is the schema?

In [ ]:
df.printSchema()

Show whole DataFrame 

In [ ]:
df.show()

Print out the first 2 rows.

In [ ]:
# Didn't strictly need a for loop, could have just then head()
for row in df.head(2):
    print(row)

Use describe() to learn about the DataFrame

In [ ]:
df.describe()

Use another data frame to learn about the statistical report

In [ ]:
temp = df.describe()
temp.show()

Get the mean of age directly

In [ ]:
from pyspark.sql.functions import mean

df.select(mean("age")).show()

What is the max and min of the Volume column?

In [ ]:
# Could have also used describe
from pyspark.sql.functions import max
from pyspark.sql.functions import min

df.select(max("age"), min("age")).show()


How many days was the age smaller than 30?

In [ ]:
df.filter("age < 30").count()

In [ ]:
from pyspark.sql.functions import count

result = df.filter(df['age'] < 30)
result.select(count('age')).show()

# PySpark Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### [Hint](https://spark.apache.org/docs/3.0.1/api/python/index.html)

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [ ]:
df = spark.read.csv('/content/walmart_stock.csv', header=True, inferSchema=True)

#### What are the column names?

In [ ]:
df.columns

#### What does the Schema look like?

In [ ]:
df.printSchema()

## Checkpoint 1:Print out the first 5 columns.

In [ ]:
for row in df.head(5):
    print(row)


## Checkpoint 2: Use describe() to learn about the DataFrame.

In [ ]:
df.describe().show()


#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](https://spark.apache.org/docs/3.0.1/api/python/pyspark.sql.html?highlight=describe#pyspark.sql.DataFrame.describe)

In [ ]:
# Uh oh Strings!
df.describe().printSchema()

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
# hint
result = df.describe()
result.select(
    result['summary'],
    format_number(result['Open'].cast('float'), 2).alias('Open'),
    format_number(result['Close'].cast('float'), 2).alias('Close'),
).show()


## Checkpoint 3: format number

In [ ]:
df.describe().select(
    result['summary'],
    format_number(result['Open'].cast('float'), 2).alias('Open'),
    format_number(result['High'].cast('float'), 2).alias('High'),
    format_number(result['Low'].cast('float'), 2).alias('Low'),
    format_number(result['Close'].cast('float'), 2).alias('Close'),
    result['Volume'].cast('int').alias('Volume'),
).show()

## Checkpoint 4: HV Ratio
HV Ratio = df["High"]/df["Volume"]
#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
df.select((df['High'] / df['Volume']).alias('HV Ratio')).show()

#### What day had the Peak High in Price?

In [ ]:
# Didn't need to really do this much indexing
# Could have just shown the entire row
df.orderBy(df["High"].desc()).head(1)[0][0]

## Checkpoint 5: What is the mean, max and min of the Close column?

In [ ]:
# Could have also used describe
from pyspark.sql.functions import max
from pyspark.sql.functions import mean
from pyspark.sql.functions import min

In [ ]:
df.select(
    mean(df['Close']),
    max(df['Close']),
    min(df['Close']),
).show()

## Checkpoint 6: How many days was the Close lower than 60 dollars?

In [ ]:
from pyspark.sql.functions import count  # hint

df.filter(df['Close'] < 60).select(count('Close')).show()

## Checkpoint 7: What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
# Many ways to do this

n_days_higher_than_80 = df.filter(df['High'] > 80).count()
total_days = df.count()
print(n_days_higher_than_80 / total_days * 100)

## Checkpoint 8: What is the Pearson correlation between High and Volume?
hint: corr("High","Volume")
#### [Hint](https://spark.apache.org/docs/3.0.1/api/python/pyspark.sql.html?highlight=dataframestatfunctions#pyspark.sql.DataFrameStatFunctions)

In [ ]:
from pyspark.sql.functions import corr  # hint

df.select(corr('High', 'Volume')).show()

#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import year

yeardf = df.withColumn("Year", year(df["Date"]))


In [ ]:
max_df = yeardf.groupBy('Year').max()
max_df.select('Year', 'max(High)').show()


## Checkpoint 9: What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
from pyspark.sql.functions import month
# hint
monthdf = df.withColumn("Month", month("Date"))


In [ ]:
# hint: group by "Month"
monthdf.groupBy('Month').avg().select('Month',
                                      'avg(Close)').sort('Month').show()


# Great!